In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler,LabelEncoder
import pickle

In [10]:
df=pd.read_csv("Churn_Modelling.csv")

In [11]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [12]:
df=df.drop(["RowNumber","CustomerId","Surname"], axis=1)

In [13]:
LB=LabelEncoder()

In [14]:
df["Gender"]=LB.fit_transform(df["Gender"])

In [15]:
df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0


In [16]:
df["Gender"].unique()

array([0, 1])

In [17]:
df["Gender"].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [22]:
one_hot=OneHotEncoder(sparse_output=False)

In [23]:
geo_encoder=one_hot.fit_transform(df[["Geography"]])

In [24]:
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [25]:
one_hot.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [26]:
df2=pd.DataFrame(geo_encoder, columns=one_hot.get_feature_names_out())

In [27]:
df2

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [28]:
df_final=pd.concat([df,df2],axis=1)

In [36]:
df_final=df_final.drop("Geography",axis=1)

In [37]:
df_final

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [38]:
X=df_final.drop("EstimatedSalary",axis=1)

In [40]:
Y=df_final["EstimatedSalary"]

In [41]:
Y

0       101348.88
1       112542.58
2       113931.57
3        93826.63
4        79084.10
          ...    
9995     96270.64
9996    101699.77
9997     42085.58
9998     92888.52
9999     38190.78
Name: EstimatedSalary, Length: 10000, dtype: float64

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [43]:
SC=StandardScaler

In [47]:
from sklearn.preprocessing import StandardScaler

SC = StandardScaler()  # or any other scaler you are using
x_train_sc = SC.fit_transform(X_train)  # Ensure X_train is defined

In [48]:
x_test_sc=SC.transform(X_test)

In [50]:
with open('Scaler.pkl','wb') as file:
    pickle.dump(SC,file)

In [51]:
with open('Label_Encoder.pkl','wb') as file:
    pickle.dump(LB,file)

    

In [52]:
with open('One_hot_encoder.pkl','wb') as file:
    pickle.dump(one_hot,file)

In [53]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [60]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(x_train_sc.shape[1],)),  # Note the comma
    Dense(32, activation='relu'),
    Dense(1)
])

In [61]:
model.compile(optimizer='adam',loss='mean_absolute_error', metrics=['mae'])

In [63]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                832       
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [65]:
## Set up the Tensorboard
import datetime
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [66]:
early_stopping=EarlyStopping(monitor='val_loss',patience=15,restore_best_weights=True)

In [67]:
history_regression=model.fit(
    x_train_sc, y_train, epochs=120, batch_size=32, validation_data=(x_test_sc,y_test), callbacks=[tensorflow_callback,early_stopping]
)

Epoch 1/120


250/250 [==============================] - 2s 4ms/step - loss: 100373.3594 - mae: 100373.3594 - val_loss: 98496.3984 - val_mae: 98496.3984
Epoch 2/120
250/250 [==============================] - 1s 2ms/step - loss: 99536.6875 - mae: 99536.6875 - val_loss: 96788.0547 - val_mae: 96788.0547
Epoch 3/120
250/250 [==============================] - 1s 2ms/step - loss: 96575.3672 - mae: 96575.3672 - val_loss: 92427.6406 - val_mae: 92427.6406
Epoch 4/120
250/250 [==============================] - 1s 3ms/step - loss: 90802.6016 - mae: 90802.6016 - val_loss: 85286.8203 - val_mae: 85286.8203
Epoch 5/120
250/250 [==============================] - 1s 2ms/step - loss: 82538.1484 - mae: 82538.1484 - val_loss: 76304.2891 - val_mae: 76304.2891
Epoch 6/120
250/250 [==============================] - 1s 2ms/step - loss: 73090.5547 - mae: 73090.5547 - val_loss: 67325.0625 - val_mae: 67325.0625
Epoch 7/120
250/250 [==============================] - 1s 2ms/step - loss: 64396.2188 - mae: 64396.218

In [72]:
pip install tensorboard

Note: you may need to restart the kernel to use updated packages.


In [73]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [75]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6007 (pid 6764), started 0:00:27 ago. (Use '!kill 6764' to kill it.)

In [76]:
test_loss,test_mae=model.evaluate(x_test_sc,y_test)

63/63 [==============================] - 0s 1ms/step - loss: 50194.5352 - mae: 50194.5352


In [78]:
model.save('regression_model.h5')

c:\Users\umair\OneDrive\Desktop\Final DL\venvDL\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
